# 🎯 OutputParsers: Transformando Respostas Bagunçadas em Dados Estruturados!

**Módulo 5 de 17 - Curso LangChain v0.3**



Fala pessoal! Pedro Guth aqui! 🚀

Tá, mas o que acontece quando você faz uma pergunta pro ChatGPT e ele te responde com um textão desorganizado? Você quer dados estruturados, mas recebe uma resposta que parece conversa de WhatsApp da família!

É aí que entram os **OutputParsers** do LangChain! Pensa neles como aquele amigo organizado que pega sua gaveta bagunçada e deixa tudo certinho, cada coisa no seu lugar.

**O que vamos ver hoje:**
- Por que precisamos de OutputParsers
- Tipos principais de parsers
- Como integrar com nossos ChatModels e PromptTemplates
- Exemplos práticos que vão te fazer falar "Liiindo!"

Bora começar!

## 🤔 O Problema: LLMs Falam Demais!

Imagina que você vai no açougue e pede "1kg de picanha". O açougueiro, em vez de te dar só a carne, começa:

*"Olha, essa picanha aqui é muito boa, vem do boi que pastava lá no sul, a textura dela é macia, eu recomendo fazer na churrasqueira com sal grosso..."*

Você só queria a picanha! 🥩

Com LLMs é igual. Você pede dados estruturados e eles vêm com:
- Explicações extras
- Formatação inconsistente
- Texto em linguagem natural

**OutputParsers resolvem isso!** Eles são como um filtro que:
1. Pega a resposta "tagarela" do LLM
2. Extrai só o que importa
3. Estrutura no formato que você precisa

### Fluxo com OutputParsers:

```mermaid
graph LR
    A[Prompt] --> B[LLM]
    B --> C[Resposta Textual]
    C --> D[OutputParser]
    D --> E[Dados Estruturados]
```

**Dica!** OutputParsers funcionam como "tradutor" entre a linguagem natural do LLM e estruturas de dados que seu código entende!

In [ ]:
# Instalando as dependências necessárias
!pip install langchain langchain-google-genai python-dotenv -q

print("Dependências instaladas! Bora pro código! 🚀")

In [ ]:
# Setup inicial - conectando com o Gemini
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate

# Carregando variáveis de ambiente
load_dotenv()

# Configurando o modelo (lembra dos módulos anteriores?)
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-exp",
    google_api_key=os.getenv("GOOGLE_API_KEY"),
    temperature=0.1  # Pouca criatividade para respostas mais consistentes
)

print("Gemini configurado! Agora vamos ver o problema sem OutputParser...")

In [ ]:
# Exemplo do PROBLEMA: sem OutputParser
prompt_sem_parser = PromptTemplate(
    input_variables=["produto"],
    template="Liste 3 características do produto {produto}"
)

# Criando a chain simples (lembra do LCEL?)
chain_sem_parser = prompt_sem_parser | llm

# Testando
resposta_bagunçada = chain_sem_parser.invoke({"produto": "smartphone"})
print("RESPOSTA SEM PARSER:")
print(resposta_bagunçada.content)
print("\n" + "="*50)
print("Tipo da resposta:", type(resposta_bagunçada.content))
print("\nViu? É só texto! Como extrair as 3 características? 🤔")

## 📋 Tipos de OutputParsers no LangChain

O LangChain tem vários tipos de parsers, cada um para uma situação específica. É como ter diferentes tipos de peneira na cozinha!



### Principais Tipos:

1. **PydanticOutputParser** 🎯
   - Para estruturas complexas definidas com Pydantic
   - Validação automática dos dados
   - Ideal para APIs e dados críticos

2. **CommaSeparatedListOutputParser** 📝
   - Transforma texto em lista separada por vírgulas
   - Simples e direto ao ponto
   - Perfeito para listas básicas

3. **StructuredOutputParser** 🏗️
   - Para estruturas simples sem Pydantic
   - Define campos manualmente
   - Meio termo entre simplicidade e estrutura

4. **JsonOutputParser** 🗂️
   - Extrai JSON válido das respostas
   - Flexível para diferentes estruturas
   - Ótimo para integrações

**Dica!** Comece sempre pelo mais simples que resolve seu problema. Não use bazuca para matar mosquito!

In [ ]:
# Importando os OutputParsers que vamos usar
from langchain.output_parsers import (
    CommaSeparatedListOutputParser,
    StructuredOutputParser,
    PydanticOutputParser
)
from langchain.schema import OutputParserException
from pydantic import BaseModel, Field
from typing import List
import json

print("OutputParsers importados! Vamos começar pelo mais simples...")

## 📝 CommaSeparatedListOutputParser: O Básico que Funciona!

Este é o mais simples de todos. É como pedir pro seu amigo listar as coisas e separar por vírgula.

**Quando usar:**
- Listas simples
- Não precisa de validação complexa
- Quer rapidez e simplicidade

**Como funciona:**
1. Parser gera instruções automáticas para o LLM
2. LLM responde seguindo o formato
3. Parser converte a resposta em lista Python

**Dica!** O parser tem um método `get_format_instructions()` que gera automaticamente as instruções para o LLM. É mágico! ✨

In [ ]:
# CommaSeparatedListOutputParser em ação!
list_parser = CommaSeparatedListOutputParser()

# Vamos ver que instruções ele gera automaticamente
print("INSTRUÇÕES AUTOMÁTICAS DO PARSER:")
print(list_parser.get_format_instructions())
print("\n" + "="*50 + "\n")

# Criando um prompt que usa essas instruções
prompt_com_parser = PromptTemplate(
    input_variables=["produto"],
    template="Liste 5 características do produto {produto}.\n\n{format_instructions}",
    partial_variables={"format_instructions": list_parser.get_format_instructions()}
)

print("Prompt criado! Vamos ver como fica...")
print(prompt_com_parser.format(produto="smartphone"))

In [ ]:
# Criando a chain completa com OutputParser
chain_com_parser = prompt_com_parser | llm | list_parser

# Testando com smartphone
resultado_lista = chain_com_parser.invoke({"produto": "smartphone"})

print("RESULTADO COM PARSER:")
print(resultado_lista)
print("\n" + "="*40)
print(f"Tipo: {type(resultado_lista)}")
print(f"Quantidade de itens: {len(resultado_lista)}")
print("\nAgora posso acessar cada item individualmente:")
for i, caracteristica in enumerate(resultado_lista, 1):
    print(f"{i}. {caracteristica.strip()}")

print("\nLiiindo! Agora temos uma lista Python de verdade! 🎉")

## 🏗️ StructuredOutputParser: Mais Organização!

Agora vamos subir um nível! O StructuredOutputParser é como ter um formulário organizado. Em vez de uma lista simples, você define campos específicos.

É tipo a diferença entre:
- **Lista simples:** "maçã, banana, laranja"
- **Estruturada:** "nome: maçã, cor: vermelha, preço: R$ 5,00"

### Como definir a estrutura:
Você cria um dicionário com:
- **Chave:** nome do campo
- **Valor:** descrição do que esperamos

**Dica!** Use este parser quando precisar de dados organizados mas não quiser a complexidade do Pydantic!

In [ ]:
# Definindo a estrutura que queremos
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

# Definindo os campos que queremos na resposta
response_schemas = [
    ResponseSchema(name="nome", description="nome do produto analisado"),
    ResponseSchema(name="preco_estimado", description="preço estimado em reais (apenas o número)"),
    ResponseSchema(name="categoria", description="categoria principal do produto"),
    ResponseSchema(name="pros", description="3 principais vantagens separadas por vírgula"),
    ResponseSchema(name="contras", description="3 principais desvantagens separadas por vírgula")
]

# Criando o parser estruturado
structured_parser = StructuredOutputParser.from_response_schemas(response_schemas)

print("INSTRUÇÕES GERADAS PELO STRUCTURED PARSER:")
print(structured_parser.get_format_instructions())
print("\nViu como ele já orienta o LLM sobre o formato JSON? Inteligente! 🧠")

In [ ]:
# Criando o prompt para análise estruturada de produto
prompt_estruturado = PromptTemplate(
    input_variables=["produto"],
    template="""Analise o produto {produto} e forneça as informações solicitadas.
    
Seja preciso e objetivo nas respostas.

{format_instructions}""",
    partial_variables={"format_instructions": structured_parser.get_format_instructions()}
)

# Criando a chain completa
chain_estruturada = prompt_estruturado | llm | structured_parser

# Testando com diferentes produtos
produtos_teste = ["iPhone 15", "Tesla Model 3", "PlayStation 5"]

for produto in produtos_teste:
    print(f"\n{'='*20} ANÁLISE: {produto} {'='*20}")
    try:
        resultado = chain_estruturada.invoke({"produto": produto})
        print(f"Nome: {resultado['nome']}")
        print(f"Preço Estimado: R$ {resultado['preco_estimado']}")
        print(f"Categoria: {resultado['categoria']}")
        print(f"Prós: {resultado['pros']}")
        print(f"Contras: {resultado['contras']}")
    except Exception as e:
        print(f"Erro ao processar {produto}: {e}")

print("\nAgora temos dados estruturados que podemos usar em qualquer aplicação! 🚀")

## 🎯 PydanticOutputParser: O Mais Profissional!

Agora vamos para o **top tier** dos OutputParsers! O PydanticOutputParser é como ter um fiscal da receita verificando seus dados - nada passa batido!



**Por que Pydantic é incrível:**
- ✅ **Validação automática** de tipos
- ✅ **Documentação automática** dos campos
- ✅ **Tratamento de erros** elegante
- ✅ **Compatibilidade** com FastAPI, SQLAlchemy, etc.

É como a diferença entre:
- **Sem validação:** Alguém te dá um papel com números
- **Com Pydantic:** Um contador certificado te entrega um relatório validado

**Dica!** Use Pydantic quando os dados forem críticos para sua aplicação. Vale o esforço extra!

In [ ]:
# Definindo um modelo Pydantic para análise de produto
from pydantic import BaseModel, Field, validator
from typing import List, Optional
from datetime import datetime

class AnaliseProduto(BaseModel):
    """Modelo para análise completa de um produto"""
    
    nome: str = Field(description="Nome completo do produto")
    categoria: str = Field(description="Categoria principal (ex: eletrônicos, veículos, games)")
    preco_min: float = Field(description="Preço mínimo estimado em reais")
    preco_max: float = Field(description="Preço máximo estimado em reais")
    nota_avaliacao: float = Field(description="Nota de 0 a 10 baseada na qualidade geral")
    características_positivas: List[str] = Field(description="Lista com 3 principais vantagens")
    características_negativas: List[str] = Field(description="Lista com 3 principais desvantagens")
    recomendado_para: str = Field(description="Perfil de usuário ideal para este produto")
    disponibilidade: str = Field(description="Facilidade para encontrar: fácil, média, difícil")
    
    # Validações customizadas
    @validator('nota_avaliacao')
    def validar_nota(cls, v):
        if not 0 <= v <= 10:
            raise ValueError('Nota deve estar entre 0 e 10')
        return v
    
    @validator('preco_min', 'preco_max')
    def validar_precos(cls, v):
        if v < 0:
            raise ValueError('Preços devem ser positivos')
        return v

# Criando o parser Pydantic
pydantic_parser = PydanticOutputParser(pydantic_object=AnaliseProduto)

print("INSTRUÇÕES GERADAS PELO PYDANTIC PARSER:")
print(pydantic_parser.get_format_instructions()[:500] + "...")
print("\nViu como ele gera instruções super detalhadas? Isso garante qualidade! 💎")

In [ ]:
# Criando prompt para análise profissional
prompt_pydantic = PromptTemplate(
    input_variables=["produto"],
    template="""Você é um especialista em análise de produtos. Analise detalhadamente o produto: {produto}

Considere:
- Preços atuais do mercado brasileiro
- Qualidade e durabilidade
- Custo-benefício
- Disponibilidade no Brasil

Seja preciso e realista nas avaliações.

{format_instructions}""",
    partial_variables={"format_instructions": pydantic_parser.get_format_instructions()}
)

# Chain completa com validação Pydantic
chain_pydantic = prompt_pydantic | llm | pydantic_parser

# Testando com um produto
produto_teste = "MacBook Air M2"
print(f"Analisando: {produto_teste}...\n")

try:
    analise = chain_pydantic.invoke({"produto": produto_teste})
    
    print(f"📱 PRODUTO: {analise.nome}")
    print(f"🏷️  CATEGORIA: {analise.categoria}")
    print(f"💰 PREÇO: R$ {analise.preco_min:,.2f} - R$ {analise.preco_max:,.2f}")
    print(f"⭐ NOTA: {analise.nota_avaliacao}/10")
    print(f"🎯 RECOMENDADO PARA: {analise.recomendado_para}")
    print(f"📦 DISPONIBILIDADE: {analise.disponibilidade}")
    
    print("\n✅ PONTOS POSITIVOS:")
    for i, ponto in enumerate(analise.características_positivas, 1):
        print(f"   {i}. {ponto}")
    
    print("\n❌ PONTOS NEGATIVOS:")
    for i, ponto in enumerate(analise.características_negativas, 1):
        print(f"   {i}. {ponto}")
    
    print(f"\n🔍 TIPO DO OBJETO: {type(analise)}")
    print("\nLiiindo! Agora temos um objeto Python completo com validação! 🎉")
    
except Exception as e:
    print(f"Erro na análise: {e}")
    print("Vamos tentar novamente com mais contexto...")

## 📊 Visualizando os Resultados

Vamos criar algumas visualizações para mostrar como os OutputParsers nos ajudam a trabalhar com dados estruturados!

**Dica!** Uma das grandes vantagens dos OutputParsers é poder usar os dados em gráficos, análises e outras aplicações!

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Vamos analisar vários produtos para criar visualizações
produtos_para_comparar = [
    "iPhone 15 Pro",
    "Samsung Galaxy S24", 
    "Google Pixel 8",
    "Xiaomi 14"
]

analises = []
nomes_produtos = []
notas = []
precos_medios = []

print("Analisando produtos para comparação...\n")

for produto in produtos_para_comparar:
    try:
        print(f"Processando {produto}...")
        analise = chain_pydantic.invoke({"produto": produto})
        
        analises.append(analise)
        nomes_produtos.append(analise.nome)
        notas.append(analise.nota_avaliacao)
        preco_medio = (analise.preco_min + analise.preco_max) / 2
        precos_medios.append(preco_medio)
        
    except Exception as e:
        print(f"Erro ao processar {produto}: {e}")

print(f"\nAnálises concluídas! {len(analises)} produtos processados.")

In [ ]:
# Criando visualizações dos dados estruturados
if len(analises) > 0:
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))
    
    # Gráfico 1: Notas de Avaliação
    colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4']
    bars1 = ax1.bar(range(len(nomes_produtos)), notas, color=colors)
    ax1.set_title('📊 Notas de Avaliação por Produto', fontsize=14, fontweight='bold')
    ax1.set_ylabel('Nota (0-10)')
    ax1.set_xticks(range(len(nomes_produtos)))
    ax1.set_xticklabels([nome.split()[-1] for nome in nomes_produtos], rotation=45)
    ax1.grid(axis='y', alpha=0.3)
    
    # Adicionando valores nas barras
    for bar, nota in zip(bars1, notas):
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
                f'{nota}', ha='center', va='bottom', fontweight='bold')
    
    # Gráfico 2: Preços Médios
    bars2 = ax2.bar(range(len(nomes_produtos)), precos_medios, color=colors)
    ax2.set_title('💰 Preços Médios por Produto', fontsize=14, fontweight='bold')
    ax2.set_ylabel('Preço (R$)')
    ax2.set_xticks(range(len(nomes_produtos)))
    ax2.set_xticklabels([nome.split()[-1] for nome in nomes_produtos], rotation=45)
    ax2.grid(axis='y', alpha=0.3)
    
    # Formatando valores monetários
    for bar, preco in zip(bars2, precos_medios):
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 100, 
                f'R$ {preco:,.0f}', ha='center', va='bottom', fontweight='bold')
    
    # Gráfico 3: Relação Preço x Nota
    scatter = ax3.scatter(precos_medios, notas, s=200, c=colors, alpha=0.7)
    ax3.set_title('💎 Custo-Benefício: Preço x Nota', fontsize=14, fontweight='bold')
    ax3.set_xlabel('Preço Médio (R$)')
    ax3.set_ylabel('Nota (0-10)')
    ax3.grid(True, alpha=0.3)
    
    # Adicionando labels nos pontos
    for i, nome in enumerate(nomes_produtos):
        ax3.annotate(nome.split()[-1], (precos_medios[i], notas[i]), 
                    xytext=(5, 5), textcoords='offset points', fontsize=10)
    
    # Gráfico 4: Categorias
    categorias = [analise.categoria for analise in analises]
    categoria_counts = {}
    for cat in categorias:
        categoria_counts[cat] = categoria_counts.get(cat, 0) + 1
    
    if categoria_counts:
        ax4.pie(categoria_counts.values(), labels=categoria_counts.keys(), 
               autopct='%1.0f%%', colors=colors[:len(categoria_counts)])
        ax4.set_title('📱 Distribuição por Categoria', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    print("Liiindo! Viu como os OutputParsers nos permitiram criar visualizações incríveis? 📊✨")
else:
    print("Não foi possível gerar gráficos - dados insuficientes.")

## 🛠️ Tratamento de Erros e Debugging

Nem sempre tudo funciona perfeitamente! LLMs podem ser imprevisíveis, igual aquele amigo que às vezes entende errado o que você falou.

![](/imagens/langchain-modulo-05_img_04.png)

**Problemas comuns:**
- LLM não segue exatamente o formato pedido
- Dados faltando ou em formato incorreto
- Caracteres especiais que quebram o JSON
- Respostas muito longas ou muito curtas

**Estratégias de recovery:**
1. **Try/catch** sempre presente
2. **Prompt engineering** mais específico
3. **Temperature baixa** para consistência
4. **Retry logic** quando necessário

**Dica!** Sempre teste seus OutputParsers com vários exemplos diferentes. Murphy's Law é real na programação!

In [ ]:
# Função helper para tentar parsing com fallback
def analisar_produto_com_retry(produto, max_tentativas=3):
    """Analisa produto com retry automático em caso de erro"""
    
    for tentativa in range(max_tentativas):
        try:
            print(f"Tentativa {tentativa + 1} para {produto}...")
            
            # Adjustando temperature baseado na tentativa
            temp = 0.1 + (tentativa * 0.1)  # Aumenta um pouco a cada tentativa
            
            llm_temp = ChatGoogleGenerativeAI(
                model="gemini-2.0-flash-exp",
                google_api_key=os.getenv("GOOGLE_API_KEY"),
                temperature=temp
            )
            
            chain_temp = prompt_pydantic | llm_temp | pydantic_parser
            resultado = chain_temp.invoke({"produto": produto})
            
            print(f"✅ Sucesso na tentativa {tentativa + 1}!")
            return resultado
            
        except OutputParserException as e:
            print(f"❌ Erro de parsing na tentativa {tentativa + 1}: {str(e)[:100]}...")
            if tentativa == max_tentativas - 1:
                print("❌ Falha após todas as tentativas. Usando fallback...")
                return None
                
        except Exception as e:
            print(f"❌ Erro geral na tentativa {tentativa + 1}: {e}")
            if tentativa == max_tentativas - 1:
                return None
    
    return None

# Testando com produtos que podem dar problema
produtos_desafiadores = [
    "Produto inexistente XYZ-2000",  # Produto que não existe
    "Tesla Cybertruck",  # Produto com info limitada
    "RTX 4090"  # Nome abreviado
]

print("Testando robustez do sistema...\n")

for produto in produtos_desafiadores:
    print(f"\n{'='*30} {produto} {'='*30}")
    resultado = analisar_produto_com_retry(produto)
    
    if resultado:
        print(f"🎉 Produto analisado: {resultado.nome}")
        print(f"📊 Nota: {resultado.nota_avaliacao}/10")
    else:
        print("💔 Não foi possível analisar este produto.")
        print("Em produção, implementaríamos um fallback aqui.")

## 🔄 Integrando com Módulos Anteriores

Lembra dos módulos que já estudamos? Vamos conectar tudo!

**Revisão rápida:**
- **Módulo 2:** ChatModel (Gemini) ✅
- **Módulo 3:** LCEL (chains com |) ✅  
- **Módulo 4:** PromptTemplate ✅
- **Módulo 5:** OutputParsers ✅ (agora!)

**Próximos módulos que se beneficiam:**
- **Módulo 6:** Chains - OutputParsers deixam chains mais poderosas
- **Módulo 10:** RAG - Parsing de documentos estruturados
- **Módulo 11:** Agents - Ferramentas que retornam dados estruturados

### Fluxo Completo:

```mermaid
graph TD
    A[Input do Usuário] --> B[PromptTemplate]
    B --> C[ChatModel - Gemini]
    C --> D[Resposta em Texto]
    D --> E[OutputParser]
    E --> F[Dados Estruturados]
    F --> G[Aplicação]
```

**Dica!** Este fluxo é a base de 90% das aplicações LangChain. Domina isso e você domina o LangChain!

In [ ]:
# Exemplo COMPLETO integrando todos os módulos anteriores
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

# Modelo mais complexo para demonstração
class ComparacaoProdutos(BaseModel):
    """Comparação detalhada entre produtos"""
    
    produto_melhor: str = Field(description="Nome do produto com melhor avaliação geral")
    produto_custo_beneficio: str = Field(description="Produto com melhor custo-benefício")
    produto_mais_caro: str = Field(description="Produto mais caro")
    resumo_comparacao: str = Field(description="Resumo em 2 frases da comparação")
    recomendacao_final: str = Field(description="Para quem você recomendaria cada produto")

# Parser para comparação
parser_comparacao = PydanticOutputParser(pydantic_object=ComparacaoProdutos)

# Prompt template mais sofisticado usando ChatPromptTemplate
prompt_comparacao = ChatPromptTemplate.from_messages([
    ("system", "Você é um especialista em análise comparativa de produtos de tecnologia."),
    ("human", """
Compare os seguintes produtos considerando:
- Preço atual no mercado brasileiro  
- Qualidade e recursos
- Custo-benefício
- Público-alvo

Produtos: {produtos}

Seja objetivo e baseado em dados reais do mercado.

{format_instructions}
    """)
])

# Chain completa usando LCEL
chain_comparacao = (
    {
        "produtos": RunnablePassthrough(),
        "format_instructions": lambda _: parser_comparacao.get_format_instructions()
    }
    | prompt_comparacao 
    | llm 
    | parser_comparacao
)

# Testando comparação
produtos_para_comparar = "iPhone 15 Pro, Samsung Galaxy S24 Ultra, Google Pixel 8 Pro"

print("🔄 Executando chain completa com todos os módulos...\n")

try:
    comparacao = chain_comparacao.invoke(produtos_para_comparar)
    
    print("🏆 RESULTADO DA COMPARAÇÃO:")
    print(f"\n🥇 Melhor Produto: {comparacao.produto_melhor}")
    print(f"💰 Melhor Custo-Benefício: {comparacao.produto_custo_beneficio}")
    print(f"💸 Mais Caro: {comparacao.produto_mais_caro}")
    print(f"\n📝 Resumo: {comparacao.resumo_comparacao}")
    print(f"\n🎯 Recomendações: {comparacao.recomendacao_final}")
    
    print("\n✨ Perfeito! Todos os módulos trabalhando juntos! ✨")
    
except Exception as e:
    print(f"Erro na comparação: {e}")
    print("Tentando novamente...")

## 🎯 Exercício Prático: Seu Primeiro Sistema Completo!

Agora é sua vez! Vamos criar um sistema de análise de filmes usando tudo que aprendemos.

**Desafio:** Criar um analisador de filmes que:
1. Receba o nome de um filme
2. Retorne análise estruturada
3. Permita comparar múltiplos filmes
4. Trate erros elegantemente

**Campos para analisar:**
- Nome do filme
- Ano de lançamento  
- Gênero principal
- Nota IMDb estimada
- Resumo em 1 frase
- 3 pontos fortes
- 2 pontos fracos
- Público recomendado

**Dica!** Use o que aprendemos: Pydantic para validação, try/catch para erros, e LCEL para chains elegantes!

In [ ]:
# EXERCÍCIO: Complete o código abaixo
# Defina o modelo Pydantic para análise de filmes

class AnaliseFilme(BaseModel):
    """Seu modelo aqui! Defina os campos necessários"""
    
    # TODO: Adicione os campos solicitados
    # Lembre-se de usar Field() com descriptions
    # E validators onde necessário (ex: ano entre 1900-2024)
    
    pass  # Remova este pass e implemente

# TODO: Crie o parser
# parser_filme = ?

# TODO: Crie o prompt template
# prompt_filme = ?

# TODO: Crie a chain completa
# chain_filme = ?

# TODO: Teste com alguns filmes
filmes_teste = ["Cidade de Deus", "Vingadores Ultimato", "Parasita"]

print("Implemente o código acima e teste! 🎬")
print("\nQuando terminar, descomente as linhas abaixo para testar:")

# for filme in filmes_teste:
#     try:
#         analise = chain_filme.invoke({"filme": filme})
#         print(f"\n🎬 {analise.nome} ({analise.ano})")
#         print(f"⭐ Nota: {analise.nota_imdb}")
#         print(f"📝 {analise.resumo}")
#     except Exception as e:
#         print(f"Erro ao analisar {filme}: {e}")

In [ ]:
# SOLUÇÃO DO EXERCÍCIO (não olhe antes de tentar!)
# Descomente para ver a solução

# class AnaliseFilme(BaseModel):
#     """Análise completa de filme"""
#     
#     nome: str = Field(description="Nome completo do filme")
#     ano: int = Field(description="Ano de lançamento")
#     genero: str = Field(description="Gênero principal (drama, ação, comédia, etc)")
#     nota_imdb: float = Field(description="Nota estimada no IMDb (0-10)")
#     resumo: str = Field(description="Resumo do filme em uma frase")
#     pontos_fortes: List[str] = Field(description="3 principais pontos fortes")
#     pontos_fracos: List[str] = Field(description="2 principais pontos fracos")
#     publico_recomendado: str = Field(description="Perfil do público ideal")
#     
#     @validator('ano')
#     def validar_ano(cls, v):
#         if not 1900 <= v <= 2024:
#             raise ValueError('Ano deve estar entre 1900 e 2024')
#         return v
#     
#     @validator('nota_imdb')
#     def validar_nota(cls, v):
#         if not 0 <= v <= 10:
#             raise ValueError('Nota deve estar entre 0 e 10')
#         return v

print("Solução comentada acima! Tente fazer primeiro! 💪")

## 🚀 Preparando para os Próximos Módulos

Parabéns! Você dominou os OutputParsers! 🎉

![](/imagens/langchain-modulo-05_img_05.png)

**O que dominamos hoje:**
- ✅ Por que OutputParsers são essenciais
- ✅ CommaSeparatedListOutputParser para listas simples
- ✅ StructuredOutputParser para dados organizados
- ✅ PydanticOutputParser para validação profissional
- ✅ Tratamento de erros e retry logic
- ✅ Integração com módulos anteriores

**Como isso se conecta com os próximos módulos:**

🔗 **Módulo 6 - Chains:** OutputParsers tornam chains muito mais poderosas
🔗 **Módulo 8 - Document Loading:** Parsing de documentos estruturados
🔗 **Módulo 10 - RAG:** Estruturação de respostas baseadas em documentos
🔗 **Módulo 11 - Agents:** Tools que retornam dados estruturados
🔗 **Módulo 14 - Deploy:** APIs que retornam JSON estruturado

**Dica Final!** OutputParsers são o "tempero" que transforma aplicações básicas em sistemas profissionais. Use sempre que precisar de dados estruturados!

## 📚 Resumo e Boas Práticas

### 🎯 Quando Usar Cada Parser:

| Parser | Quando Usar | Exemplo |
|--------|-------------|----------|
| **CommaSeparatedList** | Listas simples | "maçã, banana, laranja" |
| **Structured** | Dados organizados sem validação complexa | {"nome": "João", "idade": "30"} |
| **Pydantic** | Aplicações profissionais com validação | APIs, sistemas críticos |
| **JSON** | Estruturas flexíveis | Configurações, dados variáveis |

### 🛡️ Boas Práticas:

1. **Sempre use try/catch** - LLMs são imprevisíveis
2. **Temperature baixa** - Para respostas consistentes (0.1-0.3)
3. **Prompts específicos** - Seja claro sobre o formato desejado
4. **Validação Pydantic** - Para dados críticos
5. **Teste com dados reais** - Não apenas casos ideais
6. **Implemente retry logic** - Para aumentar robustez
7. **Use format_instructions** - Deixe o parser orientar o LLM

### 🚨 Armadilhas Comuns:

- ❌ Confiar 100% que o LLM seguirá o formato
- ❌ Não validar dados antes de usar
- ❌ Prompts muito vagos sobre formato
- ❌ Não ter fallback para erros
- ❌ Usar parser complexo para tarefa simples

### 💡 Dica de Ouro:

"OutputParsers são como contratos: definem exatamente o que você espera receber. Seja específico no que pede e generoso no que aceita como entrada!"

In [ ]:
# 🎉 CÓDIGO FINAL: Template Reutilizável
# Este é um template que você pode usar em qualquer projeto!

def criar_analisador_com_parser(modelo_pydantic, template_prompt, modelo_llm=None):
    """Factory function para criar analisadores com OutputParser
    
    Args:
        modelo_pydantic: Classe Pydantic para estruturar dados
        template_prompt: String do template do prompt
        modelo_llm: Modelo LLM (opcional, usa padrão se None)
    
    Returns:
        Chain completa pronta para uso
    """
    
    # Configuração padrão do LLM
    if modelo_llm is None:
        modelo_llm = ChatGoogleGenerativeAI(
            model="gemini-2.0-flash-exp",
            google_api_key=os.getenv("GOOGLE_API_KEY"),
            temperature=0.1
        )
    
    # Criando parser
    parser = PydanticOutputParser(pydantic_object=modelo_pydantic)
    
    # Criando prompt
    prompt = PromptTemplate(
        input_variables=["input"],
        template=template_prompt + "\n\n{format_instructions}",
        partial_variables={"format_instructions": parser.get_format_instructions()}
    )
    
    # Retornando chain completa
    return prompt | modelo_llm | parser

# Exemplo de uso do template
print("Template reutilizável criado! 🔧")
print("Use este padrão em seus projetos para ter OutputParsers consistentes!")
print("\nPróximo módulo: Chains - onde vamos combinar múltiplas operações! 🚀")
print("\nParabéns por completar o Módulo 5! Você está dominando o LangChain! 🎉")